In [1]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, 
                                names=['tuned_dlm_rm3','orignal_dlm_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20original_tuned_dlm_rm3', '20orignal_dlm_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21original_tuned_dlm_rm3', '21orignal_dlm_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
#             pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm_rm3.csv")
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/original_tuned_dlm_rm3.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.559156
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5da87a8bd0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f18 at 0x7f5da89497f0>>) fb_terms=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5da87a8bd0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f18 at 0x7f5da89497f0>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5da87a8bd0 jclass=org/terrier/querying/RM3 jself=<LocalRef

/tmp/ipykernel_3683520/1523456533.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels,


\begin{tabular}{llrrr}
\toprule
{} &             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    tuned\_dlm\_rm3 &    0.497589 &     0.363329 &  0.570722 \\
1 &  orignal\_dlm\_rm3 &    0.573641 &     0.452373 &  0.590270 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/1523456533.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20original_tuned_dlm_rm3', '20orignal_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20original\_tuned\_dlm\_rm3 &    0.451997 &     0.346067 &  0.582728 \\
1 &         20orignal\_dlm\_rm3 &    0.506382 &     0.386060 &  0.581117 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/1523456533.py:39: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21original_tuned_dlm_rm3', '21orignal_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21original\_tuned\_dlm\_rm3 &    0.542269 &     0.380245 &  0.558956 \\
1 &         21orignal\_dlm\_rm3 &    0.639554 &     0.517360 &  0.599241 \\
\bottomrule
\end{tabular}



In [4]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3', folders[i] + '_orignal_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm_rm3', '21'+ folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm_rm3.csv")
#             pipe_qe.to_csv(path_or_buf=f"/workspace/src/runs_dlm_rm3/{folders[i]}_tuned_dlm_rm3.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.518446
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d6229de50 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f78 at 0x7f5d61df6930>>) fb_terms=12', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d6229de50 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f78 at 0x7f5d61df6930>>) fb_docs=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d6229de50 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x

/tmp/ipykernel_3683520/4091487451.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3', folders[i] + '_orignal_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    conclusions\_tuned\_dlm\_rm3 &    0.443514 &     0.363653 &  0.711345 \\
1 &  conclusions\_orignal\_dlm\_rm3 &    0.524059 &     0.448322 &  0.705230 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/4091487451.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20conclusionstuned\_dlm\_rm3 &    0.393261 &     0.346387 &  0.715838 \\
1 &       20conclusions\_dlm\_rm3 &    0.441318 &     0.370764 &  0.683595 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                               name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    pools\_conclusions\_tuned\_dlm\_rm3 &    0.445642 &     0.381946 &  0.710577 \\
1 &  pools\_conclusions\_orignal\_dlm\_rm3 &    0.517417 &     0.435083 &  0.697605 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/4091487451.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20pools\_conclusionstuned\_dlm\_rm3 &    0.393539 &     0.345183 &  0.699785 \\
1 &       20pools\_conclusions\_dlm\_rm3 &    0.416576 &     0.348261 &  0.679737 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/4091487451.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm_rm3', '21'+ folders[i] + '_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21pools\_conclusionstuned\_dlm\_rm3 &    0.496703 &     0.417974 &  0.721152 \\
1 &       21pools\_conclusions\_dlm\_rm3 &    0.616240 &     0.520169 &  0.715116 \\
\bottomrule
\end{tabular}



In [5]:
import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm_rm3_for_qe = pt.BatchRetrieve(index, wmodel="DirichletLM")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = dlm_rm3_for_qe >> rm3 >> dlm_rm3_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3', folders[i] + '_orignal_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm_rm3', '21'+ folders[i] + '_dlm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm_rm3.csv")
#             pipe_qe.to_csv(path_or_buf=f"/workspace/src/runs_dlm_rm3/{folders[i]}_tuned_dlm_rm3.csv")
            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.411759
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d523fa400 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f50 at 0x7f5d615bdc10>>) fb_terms=4', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d523fa400 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f50 at 0x7f5d615bdc10>>) fb_docs=6', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d523fa400 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55768bc87f50 at 0x7f5d615bdc10>>) fb_lambda=0.2']
Fold 2
Best ndcg_cut_5 is 0.258482
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f5d523fa400 jclas

/tmp/ipykernel_3683520/313570147.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics, qrels, names=[folders[i] + '_tuned_dlm_rm3', folders[i] + '_orignal_dlm_rm3'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    marco-pretrained\_tuned\_dlm\_rm3 &    0.321715 &     0.282920 &  0.625945 \\
1 &  marco-pretrained\_orignal\_dlm\_rm3 &    0.339255 &     0.310085 &  0.608835 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/313570147.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm_rm3', '20' + folders[i] + '_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20marco-pretrainedtuned\_dlm\_rm3 &     0.25162 &     0.246023 &  0.635986 \\
1 &       20marco-pretrained\_dlm\_rm3 &     0.23545 &     0.248456 &  0.599014 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3683520/313570147.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, dlm_rm3_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm_rm3', '21'+ folders[i] + '_dlm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21marco-pretrainedtuned\_dlm\_rm3 &    0.390407 &     0.319079 &  0.616105 \\
1 &       21marco-pretrained\_dlm\_rm3 &    0.440984 &     0.370480 &  0.618459 \\
\bottomrule
\end{tabular}

